In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression

In [2]:
rand_feature_map = torch.normal(0, 1, (10, 512, 512))

In [3]:
X = rand_feature_map.numpy().reshape(-1)
y = X >= 0
X = np.clip(X + np.random.normal(0, 1, X.shape), 0, 1)

X = X[:10_000]
y = y[:10_000]

In [4]:
lr = LogisticRegression()
lr.fit(X.reshape(-1, 1), y)

ir = IsotonicRegression(out_of_bounds="clip")
ir.fit(X, y)

IsotonicRegression(out_of_bounds='clip')

In [5]:
class TorchLogisticRegression(torch.nn.Module):
    def __init__(self, lr: LogisticRegression):
        super().__init__()
        self._coef = torch.nn.Parameter(torch.from_numpy(lr.coef_))
        self._intercept = torch.nn.Parameter(torch.from_numpy(lr.intercept_))
    def forward(self, probs):
        return torch.sigmoid(probs*self._coef + self._intercept)

In [6]:
# formula: y = y1 + (x - x1)*slope

# slope[n] is for xs in interval x[n] - x[n+1]
# so formula is: y = y[n] + (x - x[n])*slopes[n]

In [7]:
class TorchIsotonicRegression(torch.nn.Module):
    def __init__(self, ir: IsotonicRegression):
        super().__init__()
        self.x_vals = torch.nn.Parameter(torch.from_numpy(ir.f_.x), requires_grad=False)
        self.y_vals = torch.nn.Parameter(torch.from_numpy(ir.f_.y), requires_grad=False)
        self.slopes = torch.nn.Parameter(
            torch.from_numpy(
                np.concatenate([
                    (ir.f_.y[1:] - ir.f_.y[:-1]) / (ir.f_.x[1:] - ir.f_.x[:-1]),
                    np.array([0.]),
                ])
        ))
    def forward(self, inputs):
        masks = []
        for x_val in self.x_vals:
            masks.append(torch.where(rand_feature_map >= x_val, x_val, 0.))
        _, ind = torch.max(torch.stack(masks, dim=0), dim=0)
        y = self.y_vals[ind] + (inputs - self.x_vals[ind]) * self.slopes[ind]
        y = torch.clamp(y, self.y_vals.min(), self.y_vals.max())
        return y

In [8]:
torchLR = TorchLogisticRegression(lr)
torchIR = TorchIsotonicRegression(ir)

### LogisticRegression Test

In [9]:
xs = rand_feature_map.numpy()
xs = xs.reshape(-1, 1)

In [10]:
%%timeit
res = lr.predict_proba(xs)[:, 1]
res = res.reshape(rand_feature_map.shape)

72.4 ms ± 15.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%%timeit
with torch.no_grad():
    res = torchLR.forward(rand_feature_map)
    _ = res.numpy()

28 ms ± 2.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
torchLR.to("cuda")
rand_feature_map = rand_feature_map.to("cuda")

In [13]:
%%timeit
with torch.no_grad():
    res = torchLR.forward(rand_feature_map)
    _ = res.cpu().numpy()

6.6 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Test Results

In [14]:
torchLR.to("cpu")
rand_feature_map = rand_feature_map.to("cpu")
with torch.no_grad():
    torch_res = torchLR.forward(rand_feature_map)
    torch_res = torch_res.numpy()

In [15]:
xs = rand_feature_map.numpy()
xs = xs.reshape(-1, 1)
sklearn_res = lr.predict_proba(xs)[:, 1]
sklearn_res = sklearn_res.reshape(rand_feature_map.shape)

In [16]:
np.allclose(torch_res, sklearn_res)

True

### IsotonicRegression Test

In [17]:
xs = rand_feature_map.numpy()
xs = xs.reshape(-1)

In [18]:
%%timeit
res = ir.predict(xs)
res = res.reshape(rand_feature_map.shape)

84.6 ms ± 13.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%%timeit
with torch.no_grad():
    res = torchIR.forward(rand_feature_map)
    _ = res.numpy()

1.23 s ± 208 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
torchIR.to("cuda")
rand_feature_map = rand_feature_map.to("cuda")

In [21]:
%%timeit
with torch.no_grad():
    res = torchIR.forward(rand_feature_map)
    _ = res.cpu().numpy()

31.3 ms ± 5.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Test Results

In [22]:
torchIR.to("cpu")
rand_feature_map = rand_feature_map.to("cpu")
with torch.no_grad():
    torch_res = torchIR.forward(rand_feature_map)
    torch_res = torch_res.numpy()

In [23]:
xs = rand_feature_map.numpy()
xs = xs.reshape(-1)
sklearn_res = ir.predict(xs)
sklearn_res = sklearn_res.reshape(rand_feature_map.shape)

In [24]:
np.allclose(torch_res, sklearn_res)

True